In [1]:
from transformers import BartTokenizer, TFBartForConditionalGeneration
from transformers.models.bart.modeling_bart import shift_tokens_right
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
# Set up the T5 tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = TFBartForConditionalGeneration.from_pretrained('facebook/bart-base')

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [3]:
df = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/quadruplet/rest15/all_data.csv')
X = df['input'].values.tolist()
y = df['quadruplet_label'].values.tolist()
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def make_data(x, y, pad_id):
    input_encodings = dict(tokenizer(x, padding=True, truncation=True, return_tensors="pt"))
    target_encodings = dict(tokenizer(y, padding=True, truncation=True, return_tensors="pt"))
    labels = target_encodings['input_ids']
    #decoder_start_token_id=0 biar <s> yang nandain start dari tokenisasi
    decoder_input_ids = shift_tokens_right(labels, pad_id, 0)
    labels = labels.masked_fill_(labels == pad_id, -100)
    encodings = {
        'input_ids': tf.convert_to_tensor(input_encodings['input_ids'].numpy()),
        'attention_mask': tf.convert_to_tensor(input_encodings['attention_mask'].numpy()),
        'decoder_input_ids': tf.convert_to_tensor(decoder_input_ids.numpy()),
        'labels': tf.convert_to_tensor(labels.numpy()),
    }
    return encodings

In [5]:
pad_id = model.config.pad_token_id

In [6]:
train_encodings = make_data(x_train, y_train, pad_id)
test_encodings = make_data(x_test, y_test, pad_id)

In [7]:
train_encodings

{'input_ids': <tf.Tensor: shape=(656, 56), dtype=int64, numpy=
 array([[    0,   133,  5765, ...,     1,     1,     1],
        [    0,   133, 28287, ...,     1,     1,     1],
        [    0,   133,  3202, ...,     1,     1,     1],
        ...,
        [    0,   133,   813, ...,     1,     1,     1],
        [    0,  2387, 25537, ...,     1,     1,     1],
        [    0, 24989,  3958, ...,     1,     1,     1]], dtype=int64)>,
 'attention_mask': <tf.Tensor: shape=(656, 56), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int64)>,
 'decoder_input_ids': <tf.Tensor: shape=(656, 77), dtype=int64, numpy=
 array([[   0,    0, 1640, ...,    1,    1,    1],
        [   0,    0, 1640, ...,    1,    1,    1],
        [   0,    0, 1640, ...,    1,    1,    1],
        ...,
        [   0,    0, 1640, ...,   

In [8]:
save_path = 'tf-bart-base-quadruplet'
checkpoint_filepath = save_path + "/" + "T5-{epoch:04d}.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

callbacks = [model_checkpoint_callback] 
metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name='accuracy') ]

In [9]:
# Define the fine-tuning parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, metrics=metrics)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [10]:
# Fine-tune the model
history = model.fit(train_encodings, epochs=50, batch_size=8, validation_data=test_encodings, callbacks=callbacks)

Epoch 1/50
82/82 [==============================] - ETA: 0s - loss: 1.5777 - accuracy: 0.0428

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0001-0.4267.ckpt\assets


INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0001-0.4267.ckpt\assets


82/82 [==============================] - 132s 1s/step - loss: 1.5777 - accuracy: 0.0428 - val_loss: 0.4267 - val_accuracy: 0.0724
Epoch 2/50
82/82 [==============================] - ETA: 0s - loss: 0.4787 - accuracy: 0.0333

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0002-0.3185.ckpt\assets


...

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0008-0.2584.ckpt\assets


82/82 [==============================] - 104s 1s/step - loss: 0.1277 - accuracy: 0.0373 - val_loss: 0.2584 - val_accuracy: 0.0835
Epoch 9/50
82/82 [==============================] - 11s 125ms/step - loss: 0.1095 - accuracy: 0.0341 - val_loss: 0.2672 - val_accuracy: 0.0726
Epoch 10/50
82/82 [==============================] - 10s 125ms/step - loss: 0.0896 - accuracy: 0.0355 - val_loss: 0.2597 - val_accuracy: 0.0726
Epoch 11/50
82/82 [==============================] - 11s 129ms/step - loss: 0.0802 - accuracy: 0.0364 - val_loss: 0.2859 - val_accuracy: 0.0817
Epoch 12/50
82/82 [==============================] - ETA: 0s - loss: 0.0885 - accuracy: 0.0333

INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0012-0.2538.ckpt\assets


INFO:tensorflow:Assets written to: tf-bart-base-quadruplet\T5-0012-0.2538.ckpt\assets


82/82 [==============================] - 109s 1s/step - loss: 0.0885 - accuracy: 0.0333 - val_loss: 0.2538 - val_accuracy: 0.0774
Epoch 13/50
82/82 [==============================] - 11s 125ms/step - loss: 0.1015 - accuracy: 0.0375 - val_loss: 0.2749 - val_accuracy: 0.0725
Epoch 14/50
82/82 [==============================] - 10s 125ms/step - loss: 0.0707 - accuracy: 0.0358 - val_loss: 0.2834 - val_accuracy: 0.0725
Epoch 15/50
82/82 [==============================] - 10s 125ms/step - loss: 0.0764 - accuracy: 0.0375 - val_loss: 0.2733 - val_accuracy: 0.0725
Epoch 16/50
82/82 [==============================] - 10s 125ms/step - loss: 0.0471 - accuracy: 0.0357 - val_loss: 0.2817 - val_accuracy: 0.0725
Epoch 17/50
82/82 [==============================] - 10s 125ms/step - loss: 0.0504 - accuracy: 0.0331 - val_loss: 0.2975 - val_accuracy: 0.0876
Epoch 18/50
82/82 [==============================] - 10s 125ms/step - loss: 0.0621 - accuracy: 0.0353 - val_loss: 0.3188 - val_accuracy: 0.0725
Epoch 

In [11]:
model.save_pretrained(save_path)

# Inference

In [12]:
import random

In [15]:
save_path = 'tf-bart-base-quadruplet'
loaded_model = TFBartForConditionalGeneration.from_pretrained(save_path)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at tf-bart-base-quadruplet.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [18]:
for i in range(5):
    idx = random.randint(0, len(x_test))
    input_text =  x_test[idx]
    encoded_query = dict(tokenizer(input_text, padding=True, truncation=True, return_tensors="tf"))
    input_ids = encoded_query["input_ids"]
    attention_mask = encoded_query["attention_mask"]
    generated_answer = loaded_model.generate(input_ids, attention_mask=attention_mask, max_length=150)
    decoded_answer = tokenizer.decode(generated_answer.numpy()[0])

    print('Masukkan: ',x_test[idx])
    print('Label sebenarnya: ',y_test[idx])
    print('Hasil prediksi model: ',decoded_answer)
    print('-'*60)

Masukkan:  In the summer months , the back garden area is really nice 
Label sebenarnya:  (back garden area, nice, POS, ambience general)
Hasil prediksi model:  </s><s>(back garden area, nice, POS, ambience general)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
------------------------------------------------------------
Masukkan:  Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening 
Label sebenarnya:  (service, great, POS, service general);(food, great, POS, food prices)
Hasil prediksi model:  </s><s>(service, great, POS, service general);(food, great at POS, food quality)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p